In [18]:
import csv
import cv2

lines = []
with open ('./data/driving_log.csv') as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        lines.append(line)

imgs = []
angles = []

for line in lines:
    source = line[0]
    filename = source.split('/')[-1]
    img_path = './data/IMG/%s' % filename
    imgs.append(cv2.imread(img_path))
    angles.append(float(line[3]))

width = imgs[0].shape[1]
height = imgs[0].shape[0]
channels = imgs[0].shape[2]
    

In [24]:
import numpy as np
X_train = np.asarray(imgs)
y_train = np.asarray(angles)

In [26]:
from skimage.exposure import equalize_hist

# Nomralize data
X_train_normalized = equalize_hist(X_train)
X_train_grayscale = np.sum(X_train_normalized/3, axis=3, keepdims=True)

In [31]:
from keras.models import Sequential
from keras.layers import Flatten, Dense

model = Sequential()
model.add(Flatten(input_shape=(160, 320, 3)))
model.add(Dense(1))

model.compile(loss='mse', optimizer='adam')
model.fit(X_train_normalized, y_train, validation_split=0.2, shuffle=True, nb_epoch=7)

model.save('model.h5')

Train on 786 samples, validate on 197 samples
Epoch 1/7
786/786 [==============================] - 1s - loss: 542.8180 - val_loss: 40.1883
Epoch 2/7
786/786 [==============================] - 1s - loss: 32.4302 - val_loss: 9.5512
Epoch 3/7
786/786 [==============================] - 1s - loss: 4.3815 - val_loss: 3.4438
Epoch 4/7
786/786 [==============================] - 1s - loss: 1.2826 - val_loss: 0.2432
Epoch 5/7
786/786 [==============================] - 1s - loss: 0.3914 - val_loss: 0.1827
Epoch 6/7
786/786 [==============================] - 1s - loss: 0.2594 - val_loss: 0.3740
Epoch 7/7
786/786 [==============================] - 1s - loss: 0.2493 - val_loss: 0.2163
